In [41]:
import os
import datetime as dt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from MoneyAfterDark import NarcoAnalytics as narc, Montana as mn, TaxTools as tax
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
slash = '/'
path = os.getcwd()
export_loc = path + slash + "Exports"

In [44]:
costs = pd.read_csv(path + slash + "spending.csv", infer_datetime_format=True)
pack = pd.read_csv(path + slash + "packages.csv", infer_datetime_format=True)
#clients = pd.read_csv(path + slash + "clients.csv", infer_datetime_format=True)
sales = pd.read_csv(path + slash + "calex.csv", infer_datetime_format=True)
x = [costs,pack,sales]
for i in x:
    narc.set_dates(i)
c = narc.split_dates(costs,format='period')
s = narc.split_dates(sales,format='period')

# App

In [47]:
app = dash.Dash(__name__, suppress_callback_exceptions=True,
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])

# Sales Layout

In [48]:
sales_layout = html.Div(children=[html.H1('Revenue Analysis',style={'textAlign':'center','color':'#503D36','font-size': 40}),
                                html.P('Analysis of Sales Packages by Month.', style={'text-align':'center', 'color':'#503D36'}),
                                html.Div([dcc.Dropdown(id='dropdown-1', options=[
                                    {'label': 'year', 'value': 'year'},
                                    {'label': 'quarter', 'value': 'quarter'},
                                    {'label': 'month', 'value': 'month'},
                                    {'label': 'week', 'value': 'week'}
                                ]),
                                          dcc.Dropdown(id='sub-dropdown-1',options=[]),
                                          dcc.Dropdown(id='sub-dropdown-2',options=[]),
                                          dcc.Dropdown(id='bar-type',options=[{'label': 'group', 'value': 'group'},
                                                                              {'label': 'stack', 'value': 'stack'}])
                                         ]),
                                html.Br(),
                                html.Br(),
                                html.Div([
                                    html.Div(dcc.Graph(id='pack-sales')),
                                    html.Div(dcc.Graph(id='cash-card')),], style={'display': 'flex'}),
                                
                                html.Div([
                                    html.Div(dcc.Graph(id='pack-tot')),
                                    html.Div(dcc.Graph(id='pay-tot'))], style={'display': 'flex'})
                               ])

@app.callback(Output('sub-dropdown-1', 'options'), [Input('dropdown-1', 'value')])
def sub_dropdown_1(main_dropdown_value):
    options = mn.input_dropdown_column_set(s,main_dropdown_value)
    return options

@app.callback(Output('sub-dropdown-2', 'options'), [Input('dropdown-1', 'value')])
def sub_dropdown_2(main_dropdown_value):
    options = mn.input_dropdown_micro(main_dropdown_value)
    return options

@app.callback([Output(component_id='pack-sales', component_property='figure'),
               Output(component_id='cash-card', component_property='figure'),
               Output(component_id='pack-tot', component_property='figure'),
               Output(component_id='pay-tot', component_property='figure')],
              [Input(component_id='dropdown-1', component_property='value'),
               Input(component_id='sub-dropdown-1', component_property='value'),
               Input(component_id='sub-dropdown-2', component_property='value'),
               Input(component_id='bar-type', component_property='value')
              ]
             )

def get_graph(main_dropdown_value,sub_dropdown_1,sub_dropdown_2,bar_mode):
    color = 'one'
    dff = s.copy()
    
    
    cat = 'package'
    order_ls = narc.column_set(dff,cat,'payment')
    order_lspt = narc.column_set(dff,'payment type','payment')
    dff = dff[dff[main_dropdown_value] == sub_dropdown_1]

    pack_sales = narc.aggregate_category(dff,sub_dropdown_2,cat,'payment',order_ls)
    cash_card = narc.aggregate_category(dff,sub_dropdown_2,'payment type','payment',order_lspt)

    mc = narc.metric_columns(pack_sales,'sum')
    pack_total = narc.graph_metrics(mc)
    ccmc = narc.metric_columns(cash_card,'sum')

    ps_fig = narc.graph_index_columns(pack_sales,colors=color,barmode=bar_mode)
    cc_fig = narc.graph_index_columns(cash_card,colors=color,barmode=bar_mode)
    pt_fig = narc.graph_metrics(mc)
    pay_tot = narc.graph_metrics(ccmc)
    
    return [ps_fig,cc_fig,pt_fig,pay_tot]

# Cost Layout

# Index

In [49]:
tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold',
    'font-family': 'garamond'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#75AEFA',
    'color': 'white',
    'padding': '6px',
    'font-family': 'garamond'
}

app.layout = html.Div([
    dcc.Tabs(id="tabs-inline", value='sales', children=[
        dcc.Tab(label='Cash Flow', value='cash-flow', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Revenue/Expenditure', value='rev-exp', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Product Sales', value='sales', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Costs', value='costs', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Tax Analysis', value='tax', style=tab_style, selected_style=tab_selected_style),
    ], style=tabs_styles),
    html.Div(id='tabs-content-inline-3')
])

@app.callback(Output('tabs-content-inline-3', 'children'),
              Input('tabs-inline', 'value'))

def render_content(tab):
    
    selected = {'cash_flow':html.H3('Cash Flow'),
                'rev-exp':html.H3('Revenue/Expenditure'),
                'sales':html.Div(sales_layout),
                'costs':html.H3('Costs'),
                'tax':html.H3('Tax')}
    
    return html.Div([selected[tab]])

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jan/2023 20:03:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 20:03:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 20:03:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 20:03:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 20:03:25] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jan/2023 20:03:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jan/2023 20:03:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\fkarim\AppData\Roaming\Python\Python39\site-packages\dash\dash.py", line 1274, in dispatch

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3629, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [27/Jan/2023 20:03:31] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/Jan/2023 20:03:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 20:03:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 20:03:47] "POST /_dash-update-component HTTP/1.1" 200 -
